In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
from elasticsearch import Elasticsearch
from elasticsearch import helpers
indexingTables = Elasticsearch()

In [2]:
def getAccuracy(idRankedTables, idQueryGoal):
    
    accuracy = 0

    for idTable in idRankedTables:
    
        if idTable == idQueryGoal:
    
            accuracy = 1
            break;

    return accuracy

In [11]:
indexingTables.indices.close(index='tables')
indexingTables.indices.put_settings(index='tables', body={"index": {"similarity": {"default": {"type": "classic"}}}})
indexingTables.indices.open(index='tables')

{'acknowledged': True, 'shards_acknowledged': True}

In [4]:
def searchIndexingMLT(query, tableField, topK):
        
    result = indexingTables.search(
    index="tables", 
    body = {
        "_source": ["tablePgID"],
        "from" : 0,
        "size" : topK,
        "query": {
            "more_like_this" : {
                "fields" : [tableField],
                "like" : "["+query+"]",
                "min_term_freq" : 0,
                "min_doc_freq":0,
                "max_query_terms":50
            }
        }
    }
    )
    
    return result

In [24]:
def searchIndexing(query, tableField, topK):
    
    result= indexingTables.search(
        index="tables", 
        body = {
        "_source": ["tablePgID"],
        "from" : 0,
        "size" : topK,
        "query": {
            "multi_match":{
              "type": "most_fields",
              "query":    query, 
              "fields": ["tablePgTitle","tableBody","tableHeader"] 
              #"fields": [tableField]
            }
        }
    })
    
    return result

In [6]:
articles = pd.read_csv('articlesDataset/articlesTestDataset', delimiter=',', header=None)
formattedArticles = articles.iloc[:,:].values

In [7]:
def executeSearch(tableField, topK):

    accuracy = []

    for articles in (formattedArticles):
    
        articleKey = articles[0]
        
        articleTitle = str(articles[1])
    
        articleText = str(articles[2])
        
        result = searchIndexing(articleTitle,tableField,topK)
    
        idRankedTables = []
    
        for hit in result['hits']['hits']:
    
            tablePgId = hit['_source']['tablePgID']
    
            idRankedTables.append(tablePgId)
    
        accuracy.append(getAccuracy(idRankedTables,articleKey))

    print(str(round(np.mean(accuracy),4))+" (±) "+str(round(np.std(accuracy),4)))

In [8]:
tableField = ["tablePgTitle","tableSectionTitle","tableCaption","tableHeader","tableBody"]
topK = [1,10,100,1000]

In [25]:
for k in topK:
    
    print("Acc@"+str(k))
    
    #for field in tableField:
        
    executeSearch(field,k)

Acc@1
0.1505 (±) 0.3576
Acc@10
0.2741 (±) 0.4461
Acc@100
0.4024 (±) 0.4904
Acc@1000
0.5339 (±) 0.4988
